pobieranie danych przez API kaggla

In [11]:
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd
import os
import keras
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras import regularizers
from sklearn.model_selection import train_test_split
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
#print(os.listdir("../input"))
import argparse

In [15]:
import keras
from keras.layers.core import Dense, Flatten, Dropout
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.applications.inception_v3 import InceptionV3
from keras.applications.xception import Xception
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg19 import VGG19

In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"paulinacieslinska","key":"8a9f1fe117227a73bb15099caabdc801"}'}

In [3]:
!pip install -q kaggle

In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [5]:
import os
import kaggle

def download_df():
    if not os.path.exists("data"):
        os.makedirs("data")
    k = kaggle.KaggleApi({"username": "jdsz5paulina", "key": "5277445bf2e6cef9aac564d8f7c5b87d"})
    k.authenticate()
    print("kaggle.com: authenticated")
    k.dataset_download_cli("grassknoted/asl-alphabet", unzip=True, path="data")
if __name__ == "__main__":
    download_df()

kaggle.com: authenticated


  0%|          | 0.00/1.03G [00:00<?, ?B/s]

100%|██████████| 1.03G/1.03G [00:16<00:00, 66.6MB/s]


In [6]:
test_data='/content/data/asl_alphabet_test/asl_alphabet_test'

In [7]:
train_data='/content/data/asl_alphabet_train/asl_alphabet_train'

In [8]:
target_size = (200,200)
target_dims = (200,200, 3) # add channel for RGB
n_classes = 29
val_frac = 0.1
batch_size = 64

data_augmentor = ImageDataGenerator(samplewise_center=True, 
                                    samplewise_std_normalization=True, 
                                    validation_split=val_frac)

train_generator = data_augmentor.flow_from_directory(train_data, target_size=target_size, batch_size=batch_size, shuffle=True, subset="training")
val_generator = data_augmentor.flow_from_directory(train_data, target_size=target_size, batch_size=batch_size, subset="validation")

Found 78300 images belonging to 29 classes.
Found 8700 images belonging to 29 classes.


In [9]:
labels_dict = {'A':0,'B':1,'C':2,'D':3,'E':4,'F':5,'G':6,'H':7,'I':8,'J':9,'K':10,'L':11,'M':12,
                   'N':13,'O':14,'P':15,'Q':16,'R':17,'S':18,'T':19,'U':20,'V':21,'W':22,'X':23,'Y':24,
                   'Z':25,'space':26,'del':27,'nothing':28}

def load_data():
    """
    Loads data and preprocess. Returns train and test data along with labels.
    """
    images = []
    labels = []
    size = 64,64
    print("LOADING DATA FROM : ",end = "")
    for folder in os.listdir(train_data):
        print(folder, end = ' | ')
        for image in os.listdir(train_data + "/" + folder):
            temp_img = cv2.imread(train_data + '/' + folder + '/' + image)
            temp_img = cv2.resize(temp_img, size)
            images.append(temp_img)
            labels.append(labels_dict[folder])
    
    images = np.array(images)
    images = images.astype('float32')/255.0
    
    labels = keras.utils.to_categorical(labels)
    
    X_train, X_test, Y_train, Y_test = train_test_split(images, labels, test_size = 0.05)
    
    print()
    print('Loaded', len(X_train),'images for training,','Train data shape =',X_train.shape)
    print('Loaded', len(X_test),'images for testing','Test data shape =',X_test.shape)
    
    return X_train, X_test, Y_train, Y_test

In [10]:
load_data()

LOADING DATA FROM : N | O | H | D | G | C | A | del | J | T | P | Q | nothing | F | S | K | E | B | X | L | V | W | I | Y | space | Z | M | U | R | 
Loaded 82650 images for training, Train data shape = (82650, 64, 64, 3)
Loaded 4350 images for testing Test data shape = (4350, 64, 64, 3)


(array([[[[0.9098039 , 0.01568628, 0.00392157],
          [0.7411765 , 0.04313726, 0.0627451 ],
          [0.72156864, 0.04313726, 0.03921569],
          ...,
          [0.7411765 , 0.05490196, 0.05098039],
          [0.7607843 , 0.05098039, 0.06666667],
          [0.85490197, 0.03921569, 0.0627451 ]],
 
         [[0.7647059 , 0.05098039, 0.05098039],
          [0.45490196, 0.4627451 , 0.41960785],
          [0.44705883, 0.47058824, 0.41960785],
          ...,
          [0.23137255, 0.21568628, 0.25882354],
          [0.25882354, 0.20784314, 0.25490198],
          [0.5254902 , 0.19607843, 0.21568628]],
 
         [[0.7647059 , 0.08235294, 0.03529412],
          [0.6392157 , 0.6666667 , 0.5647059 ],
          [0.627451  , 0.6392157 , 0.53333336],
          ...,
          [0.25882354, 0.27450982, 0.33333334],
          [0.27058825, 0.25882354, 0.32156864],
          [0.5529412 , 0.22352941, 0.24313726]],
 
         ...,
 
         [[0.7411765 , 0.02352941, 0.02352941],
          [0.15294

tworzenie modelu

In [24]:
IMG_SIZE = (200, 200)
IN_SHAPE = (*IMG_SIZE, 3)

def get_model(pretrained_model, all_character_names):
    if pretrained_model == 'inception':
        pretrained_model = InceptionV3(
            include_top=False,
            input_shape=IN_SHAPE,
            weights='imagenet'
        )
    elif pretrained_model == 'xception':
        pretrained_model = Xception(
            include_top=False,
            input_shape=IN_SHAPE,
            weights='imagenet'
        )
    elif pretrained_model == 'resnet50':
        pretrained_model = ResNet50(
            include_top=False,
            input_shape=IN_SHAPE,
            weights='imagenet'
        )
    elif pretrained_model == 'vgg19':
        pretrained_model = VGG19(
            include_top=False,
            input_shape=IN_SHAPE,
            weights='imagenet'
        )
    elif pretrained_model == 'all':
        input = Input(shape=IN_SHAPE)
        inception_model = InceptionV3(
            include_top=False,
            input_tensor=input,
            weights='imagenet'
        )
        xception_model = Xception(
            include_top=False,
            input_tensor=input,
            weights='imagenet'
        )
        resnet_model = ResNet50(
            include_top=False,
            input_tensor=input,
            weights='imagenet'
        )
        flattened_outputs = [Flatten()(inception_model.output),
                             Flatten()(xception_model.output),
                             Flatten()(resnet_model.output)]
        output = Concatenate()(flattened_outputs)
        pretrained_model = Model(input, output)
        if pretrained_model.output.shape.ndims > 2:
            output = Flatten()(pretrained_model.output)
        else:
            output = pretrained_model.output

        output = BatchNormalization()(output)
        output = Dropout(0.5)(output)
        output = Dense(128, activation='relu')(output)
        output = BatchNormalization()(output)
        output = Dropout(0.5)(output)
        output = Dense(len(all_character_names), activation='softmax')(output)
        model = Model(pretrained_model.input, output)
        for layer in pretrained_model.layers:
            layer.trainable = False
        model.summary(line_length=200)

        model.compile(optimizer='adam',
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])
        return model

In [25]:
BATCH_SIZE = 64
MODELS = {
    'inception',
    'xception',
    'resnet50',
    'vgg19',
    'all'
}

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--pretrained_model', choices=MODELS,required = True)
    parser.add_argument('--data-dir', required=True)
    parser.add_argument('--weight-directory', required=True,
                        help="Directory containing the model weight files")
    parser.add_argument('--tensorboard-directory', required=True,
                        help="Directory containing the Tensorboard log files")
    parser.add_argument('--epochs', required=True, type=int,
                        help="Number of epochs to train over.")
    args = parser.parse_args()

    tensorboard_callback = keras.callbacks.TensorBoard(
        log_dir=args.tensorboard_directory,
        histogram_freq=0,
        write_graph=True,
        write_images=False
    )
    save_model_callback = keras.callbacks.ModelCheckpoint(
        os.path.join(args.weight_directory, 'weights.{epoch:02d}.h5'),
        verbose=3,
        save_best_only=False,
        save_weights_only=False,
        mode='auto',
        period=1
    )

    data_generator = DataGenerator(args.data_dir)
    model = get_model(
        args.pretrained_model,
        data_generator.encoder.all_character_names
    )

    model.fit_generator(
        data_generator.batch_generator('train', batch_size=BATCH_SIZE),
        steps_per_epoch=200,
        epochs=args.epochs,
        validation_data=data_generator.batch_generator(
            'validation',
            batch_size=BATCH_SIZE,
            augmented=False
        ),
        validation_steps=10,
        callbacks=[save_model_callback, tensorboard_callback],
        workers=4,
        pickle_safe=True,
    )

usage: ipykernel_launcher.py [-h] --pretrained_model
                             {inception,vgg19,resnet50,all,xception}
                             --data-dir DATA_DIR --weight-directory
                             WEIGHT_DIRECTORY --tensorboard-directory
                             TENSORBOARD_DIRECTORY --epochs EPOCHS
ipykernel_launcher.py: error: the following arguments are required: --pretrained_model, --data-dir, --weight-directory, --tensorboard-directory, --epochs


SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [13]:
predicted_labels = model.predict(pixels, batch_size=1)

NameError: ignored